# 6. Intelligent Rescheduling for Disruption Repair

In the previous notebook, we explored how to handle unsatisfiable (UNSAT) problems by identifying the core conflict (MUS) and interactively relaxing constraints, which often meant dropping a task from the schedule. 

While effective, dropping tasks is often a last resort. A more flexible approach is **rescheduling**: can we find a feasible solution by not only re-allocating tasks to different teams but also by **shifting their start times**?

This notebook introduces feasibility restoration through rescheduling. We will model this as a multi-objective optimization problem where the goal is to find a new, valid schedule that is as close as possible to our original plan. We will use **Lexicographical Optimization** to explore different repair strategies by prioritizing different "similarity" objectives.

### Prerequisites

This notebook requires the `discrete-optimization` library with its `ortools` dependency.

In [ ]:
# On Colab: install the library
import sys

ON_COLAB = "google.colab" in sys.modules
if ON_COLAB:
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install discrete-optimization

### Imports

In [ ]:
from typing import Optional

import pandas as pd

from discrete_optimization.generic_tools.callbacks.callback import Callback
from discrete_optimization.generic_tools.cp_tools import ParametersCp
from discrete_optimization.generic_tools.lexico_tools import LexicoSolver
from discrete_optimization.generic_tools.result_storage.result_storage import (
    ResultStorage,
)
from discrete_optimization.workforce.generators.resource_scenario import (
    ParamsRandomness,
    generate_scheduling_disruption,
)

# Import the scheduling problem, parser, and disruption generator
from discrete_optimization.workforce.scheduling.parser import (
    AllocSchedulingProblem,
    AllocSchedulingSolution,
    get_data_available,
    parse_json_to_problem,
)

# Import the scheduling CP-SAT solver and the LexicoSolver
from discrete_optimization.workforce.scheduling.solvers.cpsat import (
    CPSatAllocSchedulingSolver,
    ObjectivesEnum,
)

# Import utilities for visualization and analysis
from discrete_optimization.workforce.scheduling.utils import (
    compute_changes_between_solution,
    plotly_schedule_comparison,
)

### Setup the Disruption Scenario

As before, we'll start by creating an optimal baseline plan and then introduce a disruption that makes it infeasible. This time, we will work directly with the `AllocSchedulingProblem`, where task start times are not fixed.

In [ ]:
# Load the original scheduling problem
files = get_data_available()
file_path = files[1]
problem_original = parse_json_to_problem(file_path)

# 1. Solve the original problem to get a baseline optimal solution
print("Solving the original problem to get a baseline...")
solver_original = CPSatAllocSchedulingSolver(problem_original)
solver_original.init_model(objectives=[ObjectivesEnum.NB_TEAMS])
result_original = solver_original.solve(time_limit=10)
sol_original, fit_original = result_original.get_best_solution_fit()
fits = problem_original.evaluate(sol_original)
print(f"Original optimal solution found with {fits['nb_teams']} teams.")

# 2. Generate a disruption scenario based on this solution
print("\nGenerating a disruption...")
disruption_scenario = generate_scheduling_disruption(
    original_scheduling_problem=problem_original,
    original_solution=sol_original,
    params_randomness=ParamsRandomness(
        upper_nb_disruption=1,
        lower_nb_teams=1,
        upper_nb_teams=3,
        duration_discrete_distribution=([60], [1.0]),
    ),
)
problem_disrupted = disruption_scenario["scheduling_problem"]
additional_constraints = disruption_scenario["additional_constraint_scheduling"]
print("Disrupted problem created.")

# 3. Plot the original, now-infeasible plan for reference
print("\nOriginal plan (now likely infeasible due to disruption):")
plotly_schedule_comparison(
    sol_original,
    sol_original,
    problem_disrupted,
    title="Original Plan (Now Infeasible)",
    display=False,
    plot_team_breaks=True,
)

## Similarity Objectives & Lexicographical Optimization

To repair the schedule, we want to find a new feasible plan that is as close as possible to the original. We can measure this "closeness" using several **similarity objectives**:

- **`nb_not_done`**: Minimize the number of tasks that are dropped from the schedule. This is usually the highest priority.
- **`reallocated`**: Minimize the number of tasks that are assigned to a different team.
- **`nb_shift`**: Minimize the number of tasks that change their start time.
- **`sum_delta_schedule`**: Minimize the total duration of all time shifts (e.g., shifting two tasks by 5 min is better than shifting one by 15 min).
- **`max_delta_schedule`**: Minimize the largest time shift for any single task.

These objectives are often conflicting. **Lexicographical Optimization** is a technique to handle this by defining a strict priority order. The solver first finds the best possible value for the highest-priority objective. It then adds that value as a hard constraint and moves on to optimize the second-priority objective, and so on. 

By changing the order of these objectives, we can generate different, equally valid repair strategies.

### The `Callback` for Warm Start

To speed up the solving process, we'll use a `Callback`. At the end of each step in the lexicographical optimization, this callback will retrieve the solution found and use it as a starting point (*warm start*) for the next step. This guides the solver and can significantly reduce search time.

In [ ]:
class LexicoCpsatPrevStartCallback(Callback):
    """Callback to use the solution of a lexico-step as a warmstart for the next one."""

    def on_step_end(
        self, step: int, res: ResultStorage, solver: LexicoSolver
    ) -> Optional[bool]:
        subsolver: CPSatAllocSchedulingSolver = solver.subsolver
        # The subsolver has an internal mechanism to store the last solution
        # and use it as a hint for the next solve.
        subsolver.set_warm_start_from_previous_run()
        return False  # Never stop the lexico solver early

### Strategy A: Prioritize Minimizing Team Reallocations

This strategy is useful if changing a task's team is very costly (e.g., requires training, travel). We prefer to shift tasks in time rather than reassigning teams.

The process follows these steps:
1. Define the lexicographical order of objectives.
2. Initialize the underlying CP-SAT solver, making sure it is aware of *all* the objectives we might want to use in our sequence (e.g., `NB_DONE_AC`, `DELTA_TO_EXISTING_SOLUTION` which initializes the reallocated/nb_shift etc).
3. Initialize the `LexicoSolver` with this pre-configured subsolver.
4. Solve the problem using the defined objective order and the warm-start callback.

In [ ]:
# 1. Define the objective order: first reallocations, then shifts.
objectives_realloc_first = [
    ObjectivesEnum.NB_DONE_AC,  # Maximize completed tasks
    "reallocated",  # Minimize reallocations
    "nb_shifted",  # Minimize number of shifted tasks
    "sum_delta_schedule",  # Minimize sum of shifts
    "max_delta_schedule",  # Minimize max shift
    ObjectivesEnum.NB_TEAMS,  # Finally, minimize number of teams
]

# 2. Set up the subsolver with all possible similarity objectives
solver_realloc = CPSatAllocSchedulingSolver(problem_disrupted)
solver_realloc.init_model(
    objectives=[
        ObjectivesEnum.NB_DONE_AC,
        ObjectivesEnum.DELTA_TO_EXISTING_SOLUTION,
        ObjectivesEnum.NB_TEAMS,
    ],
    additional_constraints=additional_constraints,
    optional_activities=True,  # Allows tasks to be dropped
    base_solution=sol_original,  # Needed to calculate deltas
)
# 3. Initialize the LexicoSolver with the warm start callback
lexico_solver_A = LexicoSolver(subsolver=solver_realloc, problem=problem_disrupted)
callbacks = [LexicoCpsatPrevStartCallback()]
p = ParametersCp.default_cpsat()
p.nb_process = 12
# 4. Solve the problem
result_realloc = lexico_solver_A.solve(
    callbacks=callbacks,
    objectives=objectives_realloc_first,
    parameters_cp=p,
    time_limit=5,  # 5 seconds for each step in the lexico chain
    ortools_cpsat_solver_kwargs={"log_search_progress": False},
)
sol_realloc_first = result_realloc[-1][0]

In [ ]:
# Analyze and display the result
changes_realloc = compute_changes_between_solution(sol_original, sol_realloc_first)
print("--- Strategy A: Reallocation First ---")
for key in ["nb_reallocated", "nb_shift", "sum_shift", "max_shift"]:
    print(f"{key}: {changes_realloc[key]}")

plotly_schedule_comparison(
    sol_original,
    sol_realloc_first,
    problem_disrupted,
    title="Repaired Plan: Prioritizing Fewest Reallocations",
    display=False,
    plot_team_breaks=True,
)

### Strategy B: Prioritize Minimizing Time Shifts

Now, let's try the opposite. We'll prioritize keeping tasks at their original times, even if it means more teams need to be reassigned. This is useful when temporal constraints are very tight.

In [ ]:
# 1. Define the new objective order: first shifts, then reallocations.
objectives_shift_first = [
    ObjectivesEnum.NB_DONE_AC,
    "nb_shifted",
    "sum_delta_schedule",
    "max_delta_schedule",
    "reallocated",
    ObjectivesEnum.NB_TEAMS,
]

# 2. Set up a new subsolver instance for this strategy
solver_shift = CPSatAllocSchedulingSolver(problem_disrupted)
solver_shift.init_model(
    objectives=[
        ObjectivesEnum.NB_DONE_AC,
        ObjectivesEnum.DELTA_TO_EXISTING_SOLUTION,
        ObjectivesEnum.NB_TEAMS,
    ],
    additional_constraints=additional_constraints,
    optional_activities=True,
    base_solution=sol_original,
)

# 3. Initialize the LexicoSolver
lexico_solver_B = LexicoSolver(subsolver=solver_shift, problem=problem_disrupted)
p = ParametersCp.default_cpsat()
p.nb_process = 12
# 4. Solve
result_shift = lexico_solver_B.solve(
    callbacks=callbacks,  # Reuse the same warm start callback
    objectives=objectives_shift_first,
    parameters_cp=p,
    time_limit=5,
    ortools_cpsat_solver_kwargs={"log_search_progress": False},
)
sol_shift_first = result_shift[-1][0]

In [ ]:
# Analyze and display the result
changes_shift = compute_changes_between_solution(sol_original, sol_shift_first)
print("--- Strategy B: Shift First ---")
for key in ["nb_reallocated", "nb_shift", "sum_shift", "max_shift"]:
    print(f"{key}: {changes_shift[key]}")

plotly_schedule_comparison(
    sol_original,
    sol_shift_first,
    problem_disrupted,
    title="Repaired Plan: Prioritizing Fewest Time Shifts",
    display=False,
    plot_team_breaks=True,
)

## Analysis and Conclusion

Let's compare the outcomes of our two different repair strategies in a summary table.

In [ ]:
summary_data = {
    "Strategy A (Realloc First)": changes_realloc,
    "Strategy B (Shift First)": changes_shift,
}

summary_df = pd.DataFrame(summary_data).T[
    ["nb_reallocated", "nb_shift", "sum_shift", "max_shift"]
]
display(summary_df)

## BONUS more systematic optim
As in the CP-25 paper soon to be published, we can consider even more ordering of objectives to propose to the planner, like minimizing sum of shift or maximum shifts first. 

In [ ]:
class LexicoCpsatPrevStartCallback(Callback):
    """Callback to use the solution of a lexico-step as a warmstart for the next one."""

    def on_step_end(
        self, step: int, res: ResultStorage, solver: LexicoSolver
    ) -> Optional[bool]:
        subsolver: CPSatAllocSchedulingSolver = solver.subsolver
        subsolver.set_warm_start_from_previous_run()
        return False  # Never stop the lexico solver early


def run_lexico_strategy(
    problem: AllocSchedulingProblem,
    base_solution: AllocSchedulingSolution,
    constraints: list,
    objectives: list[str],
    strategy_name: str,
):
    """Runs a lexico-optimization strategy and returns results."""
    print(f"--- Running Strategy: {strategy_name} ---")

    # Set up the subsolver with all possible objectives it might need
    subsolver = CPSatAllocSchedulingSolver(problem)
    subsolver.init_model(
        objectives=[
            ObjectivesEnum.NB_DONE_AC,
            ObjectivesEnum.DELTA_TO_EXISTING_SOLUTION,
            ObjectivesEnum.NB_TEAMS,
        ],
        additional_constraints=constraints,
        optional_activities=True,
        base_solution=base_solution,
    )

    # Use the warm-start callback
    callbacks = [LexicoCpsatPrevStartCallback()]
    p = ParametersCp.default_cpsat()
    p.nb_process = 12
    # Create and run the lexico solver
    lexico_solver = LexicoSolver(subsolver=subsolver, problem=problem)
    result = lexico_solver.solve(
        callbacks=callbacks,
        objectives=objectives,
        time_limit=5,
        ortools_cpsat_solver_kwargs={"log_search_progress": False},
    )

    # Get results
    repaired_solution = result[-1][0]
    changes = compute_changes_between_solution(base_solution, repaired_solution)

    # Create visualization
    fig = plotly_schedule_comparison(
        sol_original,
        repaired_solution,
        problem,
        title=f"Repaired Plan - Strategy: {strategy_name}",
        display=False,
        plot_team_breaks=True,
    )

    return repaired_solution, changes, fig

In [ ]:
# Strategy A: Reallocation first
objectives_A = [
    ObjectivesEnum.NB_DONE_AC,
    "reallocated",
    "nb_shifted",
    "sum_delta_schedule",
    "max_delta_schedule",
    ObjectivesEnum.NB_TEAMS,
]
sol_A, changes_A, fig_A = run_lexico_strategy(
    problem_disrupted,
    sol_original,
    additional_constraints,
    objectives_A,
    "Reallocation First",
)

# Strategy B: Shift first
objectives_B = [
    ObjectivesEnum.NB_DONE_AC,
    "nb_shifted",
    "sum_delta_schedule",
    "max_delta_schedule",
    "reallocated",
    ObjectivesEnum.NB_TEAMS,
]
sol_B, changes_B, fig_B = run_lexico_strategy(
    problem_disrupted, sol_original, additional_constraints, objectives_B, "Shift First"
)

# Strategy C: Sum of shifts first
objectives_C = [
    ObjectivesEnum.NB_DONE_AC,
    "sum_delta_schedule",
    "max_delta_schedule",
    "nb_shifted",
    "reallocated",
    ObjectivesEnum.NB_TEAMS,
]
sol_C, changes_C, fig_C = run_lexico_strategy(
    problem_disrupted,
    sol_original,
    additional_constraints,
    objectives_C,
    "Sum-Shift First",
)

# Strategy D: Max-shift first
objectives_D = [
    ObjectivesEnum.NB_DONE_AC,
    "max_delta_schedule",
    "sum_delta_schedule",
    "nb_shifted",
    "reallocated",
    ObjectivesEnum.NB_TEAMS,
]
sol_D, changes_D, fig_D = run_lexico_strategy(
    problem_disrupted,
    sol_original,
    additional_constraints,
    objectives_D,
    "Max-Shift First",
)

In [ ]:
print("--- Summary of All Strategies ---")

# Table View
summary_data = {
    "A (Realloc First)": changes_A,
    "B (Shift First)": changes_B,
    "C (Sum-Shift First)": changes_C,
    "D (Max-Shift First)": changes_D,
}
summary_df = pd.DataFrame(summary_data).T[
    ["nb_reallocated", "nb_shift", "sum_shift", "max_shift"]
]
display(summary_df)

# Visual Comparison
print("\n--- Visual Comparison of Key Strategies ---")
fig_A.show()
fig_B.show()
fig_C.show()
fig_D.show()

By looking at the results, you can clearly see the trade-off. **Strategy A** resulted in fewer team reallocations but had to shift more tasks (and by a larger amount). **Strategy B** kept the schedule very stable in time (`nb_shift` is low) but at the cost of reassigning many more tasks to different teams.

This notebook has shown how to move beyond simple feasibility checking. By combining rescheduling with a multi-objective lexicographical approach, we can provide planners with a set of intelligent, alternative repair strategies. This empowers them to make informed decisions that balance competing operational priorities when faced with inevitable, real-world disruptions.

This concludes our series on the Team Allocation and Scheduling problem. We hope you've enjoyed the journey from problem definition to advanced, interactive decision support!